In [ ]:
# !pip install pyamg
# import pyamg
import sys
import cv2
import time
import glob
import scipy.sparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image 
from torch import optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd.variable import Variable
from torchvision import datasets, transforms

In [ ]:
device='cpu'
BATCH_SIZE =1

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', 500)

def gen_mask(sample_batch, img_size):
    np.random.seed(seed=int(time.time())) 
    masks = np.ones((sample_batch, img_size, img_size), dtype=np.float32)
    scale = 0.25
    low, upper = int(img_size * scale), int(img_size * (1.0 - scale))
    masks[:, int(low*2.7):int(upper*1.2), low:upper] = 0.
    return masks


mask = gen_mask(1,23)
print(str(mask))

In [ ]:
def rescale(image,image_size):
    rescaled = np.resize(image,(image_size,image_size))
    return torch.from_numpy(rescaled)

In [ ]:
folder_data = glob.glob("../input/beck-inpainting/WhatsApp Image 2021-06-01 at 12.48.32.jpeg")
len_data = len(folder_data)
print(len_data)
# i=4
# test_image_paths = folder_data[i:BATCH_SIZE+i]
test_image_paths = folder_data[0:BATCH_SIZE]


# folder_data = glob.glob("../input/ffhq-face-data-set/thumbnails128x128/*.png")
# folder_data = glob.glob("../input/matlam-imgs/*.jpg")

size_of_train =len(folder_data)
test_image_paths = folder_data[size_of_train-16:size_of_train]

print(len(test_image_paths))

class TestDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.transforms = transforms.Compose([
                                   transforms.Resize(150),
                                   transforms.CenterCrop(128),
                                   transforms.ToTensor(),
#             0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
            
                                  transforms.Normalize((0.2, 0.2, 0.2), (0.5, 0.5, 0.5)),
            
                               ])

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        mask = gen_mask(1,128)
        t_image = self.transforms(image)
#         t_mask = self.scale(mask,64)
        return t_image

    def __len__(self):
        return len(self.image_paths)

In [ ]:
test_dataset = TestDataset(test_image_paths)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
print(len(test_loader))
import matplotlib.pyplot as plt
print(BATCH_SIZE)
for j, inputs in enumerate(test_loader):
    for i in range(BATCH_SIZE):
        plt.subplot(np.sqrt(BATCH_SIZE), np.sqrt(BATCH_SIZE), 1 + i)
        plt.axis('off')
        plt.imshow(np.transpose(inputs[i],(1,2,0)))

In [ ]:
def get_weights(masks,window_size):
    assert (window_size+1)%2 == 0
    pad_value = (window_size-1)/2
    padded_masks = F.pad(1-masks, (int(pad_value),int(pad_value),int(pad_value),int(pad_value)), "constant", 0)
    padded_masks.unsqueeze_(1)
    kernel = torch.ones((window_size,window_size))
    kernel = kernel.view(1, 1, window_size, window_size)
    kernel = kernel.to(device)
    temp = F.conv2d(padded_masks,kernel)
    temp.squeeze_(1)
    weights = (1/window_size)*torch.mul(temp,masks)
    return weights

def context_loss(outputs,images):
    batch_size = images.size()[0]
#     W = get_weights(masks,window_size)
    W = gen_mask(1,128)
    W = np.tile(W,(3,1,1))
    W = torch.from_numpy(W)
    W.unsqueeze_(1)

    print(W.shape)
    plt.imshow(torch.mul(outputs-images,W).cpu().detach().numpy())

#     W.unsqueeze_(1)
#     W = torch.tensor(W)
#     W = W.to(device)

#     W = torch.cat((W,W,W),dim=1)
#     torch.mul(outputs-images,W)
#     plt.imshow(torch.mul(outputs-images,W).cpu().detach().numpy())
#     closs = (1/batch_size)*(torch.sum(torch.abs(torch.mul(outputs-images,W))))
    closs = (1/batch_size)*(torch.sum(torch.abs((outputs-images))))

    return closs

def prior_loss(discriminator_outputs,Lambda):
    batch_size = discriminator_outputs.size()[0]
#     check accordign to the article
    ones  = torch.ones_like(discriminator_outputs)
    ploss = (1/batch_size)*(Lambda*torch.sum(torch.log(ones - discriminator_outputs)))
    return ploss

def random_z(batch_size):
#     Z = torch.randn((batch_size,100,1,1), requires_grad=True,device="cuda")
    Z = torch.randn((batch_size,100,1,1), requires_grad=True)
    return Z

In [ ]:
class Generator(torch.nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 1024, kernel_size = 4, stride = 1, padding = 0, bias = False),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace = True),

            nn.ConvTranspose2d(1024, 512, kernel_size = 4, stride = 2, padding = 1, bias =False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace = True),

            nn.ConvTranspose2d(512, 256, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace = True),

            nn.ConvTranspose2d(256, 128, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),
            
            nn.ConvTranspose2d(128, 128, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),

            nn.ConvTranspose2d(128, 3, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.Tanh()
        )
        
    def forward(self, x):
            x = self.main(x)
            return x

generator = Generator()
generator.float()
# generator = generator.to(device)
print(generator)


In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
          nn.Conv2d(3, 128, kernel_size = 5, stride = 2, padding = 2, bias = False),
          nn.LeakyReLU(0.2, inplace=True),

          nn.Conv2d(128, 128, kernel_size = 5, stride = 2, padding = 2, bias = False),
          nn.BatchNorm2d(128),
          nn.LeakyReLU(0.2, inplace=True),

          nn.Conv2d(128, 256, kernel_size = 5, stride = 2, padding = 2, bias = False),
          nn.BatchNorm2d(256),
          nn.LeakyReLU(0.2, inplace=True),

          nn.Conv2d(256, 512, kernel_size = 5, stride = 2, padding =2, bias = False),
          nn.BatchNorm2d(512),
          nn.LeakyReLU(0.2, inplace=True),

          nn.Conv2d(512, 1024, kernel_size = 5, stride = 2, padding = 2, bias = False),
          nn.BatchNorm2d(1024),
          nn.LeakyReLU(0.2, inplace=True),

          nn.Conv2d(1024, 1, kernel_size = 4, stride = 1, padding = 0, bias = False)
        )
    
    def forward(self, x):
        x = self.main(x)
        return x

discriminator = Discriminator()
discriminator.float()
# discriminator = discriminator.to(device)
print(discriminator)


In [ ]:
generator = Generator()
generator.load_state_dict(torch.load("../input/gan128/g2_epoch-25.pth", map_location=torch.device('cpu') ))
# generator.load_state_dict(torch.load("../input/gan-64/g_celeb_epoch-40.pth"))

generator = generator.to(device)
generator.eval()
for param in generator.parameters():
    param.requires_grad = False

discriminator = Discriminator()
discriminator.load_state_dict(torch.load("../input/gan128/d2_epoch-25.pth", map_location=torch.device('cpu') ))
# discriminator.load_state_dict(torch.load("../input/gan-64/d_celeb_epoch-40.pth"))

discriminator = discriminator.to(device)

discriminator.eval()
for param in discriminator.parameters():
    param.requires_grad = False

In [ ]:
from tqdm import notebook,tqdm


In [ ]:
import matplotlib.pyplot as plt
for j, inputs in enumerate(test_loader):
    for i in range(BATCH_SIZE):
        plt.subplot(4, 4, 1 + i)
        plt.axis('off')
        plt.imshow(np.transpose(inputs[i],(1,2,0)))


In [ ]:
for i, images in tqdm(enumerate(test_loader), total=len(test_loader)):
    losses = []
    images = images.to(device)
    Z = random_z(batch_size=BATCH_SIZE)
    optimizer = optim.Adam([Z])
    if (i+1)%10==0 or i==0:
        print("Starting Training for Batch "+str(i+1)+"...")
    for iter in notebook.tqdm(range(8000)):
        outputs = generator(Z)
#         todo connect the outputs and the images
        discriminator_outputs = discriminator(outputs)
#         todo give more weight to the place without the mask
        closs = context_loss(outputs,images)
#         ploss = prior_loss(discriminator_outputs,Lambda=0.003)
        ploss = prior_loss(discriminator_outputs,Lambda=20.003)
        inpainting_loss = closs + ploss
        optimizer.zero_grad()
        inpainting_loss.backward()
        optimizer.step()
        losses.append(inpainting_loss)
        if (i+1)%10==0 or i==0:
            if (iter+1)%500==0:
                print("Iteration No. = "+str(iter+1))
                print("Loss = "+str(inpainting_loss))
    if (i+1)%10==0 or i==0:
        plt.figure(figsize=(10,5))
        plt.title("Inpainting Loss During Training")
        plt.plot(losses)
        plt.xlabel("iterations")
        plt.ylabel("Loss")
        plt.show()
    Z_optimal = Z.detach()
#     generated_images = generator(Z_optimal)
#     generated_images = generated_images.detach().cpu().numpy()
#     generated_images = np.transpose(generated_images,(0,2,3,1))
#     generated_images = ((generated_images+1.)/2.)*255.
#     generated_images = generated_images.astype(np.uint8)
    
    with torch.no_grad():
        generated_images = generator(Z_optimal.detach())

    for i in range(BATCH_SIZE):
        plt.subplot(4, 4, 1 + i)
#         plt.subplot(1, 1, 1 + i)
        plt.axis('off')
        plt.imshow(np.transpose(generated_images.cpu().numpy()[i],(1,2,0)))
    plt.show()  


#     for j in range(generated_images.shape[0]):
#         inpainted_image = Image.fromarray(np.uint8(generated_images))
#         inpainted_image.save("{}Inpainted_Image.png".format(1+(i*generated_images.shape[0])+j),"PNG")

In [ ]:
with torch.no_grad():
    generated_images = generator(Z_optimal.detach())

i=0
for j, inputs in enumerate(test_loader):
    plt.imshow(np.transpose(generated_images.cpu().numpy()[j+i],(1,2,0)))
    plt.show()  
    plt.imshow(np.transpose(inputs[j+i],(1,2,0)))
    plt.show()  

# for i in range(BATCH_SIZE):
#     plt.subplot(4, 1, 1 + i)
#     plt.axis('off')
#     plt.imshow(np.transpose(generated_images.cpu().numpy()[i],(1,2,0)))
# plt.show()  


In [ ]:
import matplotlib.pyplot as plt
for j, inputs in enumerate(test_loader):
    for i in range(BATCH_SIZE):
        plt.subplot(1, 1, 1 + i)
        plt.axis('off')
        plt.imshow(np.transpose(inputs[i],(1,2,0)))

#      plt.figure()
#      plt.imshow(inputs[i].numpy())
#      plt.show()
# images, labels = dataiter.next()
# plt.imshow(np.transpose(images[0].numpy(), (1, 2, 0)))

# for i in range(16):
#     plt.subplot(4, 4, 1 + i)
#     plt.axis('off')
#     plt.imshow(np.transpose(train_np[i],(1,2,0)))
# plt.show()  
